# Track cyclones with TempestExtremes within JASMIN Jupyterhub
TempestExtremes is a C++ software for detecting and manipulating features in climate data.  
The Software is described two papers:
* Ullrich, P.A., C.M. Zarzycki, E.E. McClenny, M.C. Pinheiro, A.M. Stansfield and K.A. Reed (2021) "TempestExtremes v2.1: A community framework for feature detection, tracking and analysis in large datasets" Geosci. Model. Dev. 14, pp. 5023–5048, doi: 10.5194/gmd-14-5023-2021.
* Ullrich, P.A. and C.M. Zarzycki (2017) "TempestExtremes v1.0: A framework for scale-insensitive pointwise feature tracking on unstructured grids" Geosci. Model. Dev. 10, pp. 1069-1090, doi: 10.5194/gmd-10-1069-2017.

And you can find the documentation here: Please find documentation here: https://climate.ucdavis.edu/tempestextremes.php

### To do in terminal before the notebook
A. If you don't already have a dedicated environment.
1. Create a conda environment
```
conda create -n tempestextremes
conda init
bash
conda activate tempestextremes
```
2. Install kernels to work with the notebooks
```
conda install ipykernels
python -m ipykernel install --user --name=tempestextremes
conda install bash_kernel
```
3. Restart jupyterhub
4. (Still in terminal) Install TempestExtremes in the environment
```
conda init
bash
conda activate tempestextremes
conda install -c conda-forge tempest-extremes
```

B. If you already created a hackathon-specific conda environment (following e.g. https://digital-earths-global-hackathon-uk.github.io/#software-stack)
1. `conda activate hackathon`
2. `python -m ipykernel install --user --name=name-of-environment` (If not already done)
3. `conda install bash kernels`

### This notebook
NB: Open this notebook with bash kernel. 

In [ ]:
# Activate the conda environment in which tempestextremes has been install
conda activate tempestextremes
# If you have a message saying you need to initalize conda: 
## create a new cell with the `conda init` command
## Run that cell
## Delete the cell
## Restart your kernel
## Run the notebook again

To track cyclone with TempestExtremes, you need to first run `DetectNodes` to find suitable "candidate nodes" (points in space and time that could be cyclones), and second run `StitchNodes` to "Stitch" the candidate nodes into tracks. In this notebook, we show some minimal examples of how that works, and then provide the code for the UZ (Ullrich & Zarzycki, sometimes also known as the eponymous "TempestExtremes").

## Tutorial

### Minimal `DetectNodes` command

In [ ]:
# Here we only find SLP minima in one file
# The file contains ERA5 SLP for one time step
DetectNodes \
--in_data "/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_199611180000.msl.nc" \
--out nodes.txt \
--searchbymin "msl" \
--latname "latitude" --lonname "longitude" 
# The output will summarize command arguments, and give you information about the process.

In [ ]:
# Visualize nodes.txt output file
head nodes.txt
# For each time step, list of candidate nodes 
# For each candidate node, contains lon. index, lat. index, lon. value and lat. value
# Will contain more columns if more info is requested through the `outputcmd` argument

In [ ]:
# Clean 
rm nodes.txt

### `DetectNodes` over several files
#### Several file of the same variable at different times
If you want to search through several files at once, you need to create a text file listing the file paths, and then provide it to `--in_data_list`.

In [ ]:
ls /badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/*msl* > flist.txt

In [ ]:
# Create folder to store nodes in
if ! [ -e nodes ]
then
    mkdir nodes
else 
    echo "nodes folder already exist, watch out for potential collisions."
fi

In [ ]:
DetectNodes \
--in_data_list flist.txt \
--out nodes/ \
--searchbymin "msl" \
--latname "latitude" --lonname "longitude" 
rm log* # Comment this if you need the log files for debugging

In [ ]:
ls nodes

In [ ]:
# Each file in nodes contains the list of candidate nodes for one input file
head nodes/000000.dat

In [ ]:
# Clean
rm -rf nodes flist.txt

#### Several files of different variables at the same time
The input filelist must contain all files path separated by a `;`.

In [ ]:
# Here we take msl, 10u and 10v files for a signle time step
msl_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_199611180000.msl.nc"
u10_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_199611180000.10u.nc"
v10_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_199611180000.10v.nc"
echo ${msl_file}\;${u10_file}\;${v10_file} > flist.txt

In [ ]:
# Note there is now a new lines "outputcmd" which specifies which information to add to output
DetectNodes \
--in_data_list flist.txt \
--out nodes.txt \
--searchbymin "msl" \
--outputcmd "msl,min,0;_VECMAG(u10,v10),max,2" \
--mergedist "6.0" \
--latname "latitude" --lonname "longitude" 

In [ ]:
# Nodes file now contains two more coluns, with the SLP minimum and the 10m wind maximum
head nodes.txt

In [ ]:
# Clean
rm nodes.txt flist.txt

#### Several files of different variables over several times

In [ ]:
# Create folder to store nodes in
if ! [ -e nodes ]; then mkdir nodes; 
else echo "nodes folder already exist, watch out for potential collisions."; 
fi
# Create folder to store logs in
if ! [ -e logs ]; then mkdir logs; 
else echo "logs folder already exist, watch out for potential collisions.";
fi

In [ ]:
# Loop over time (in that case hours in a day)
for h in 00 01 02 03 04
do 
    echo $h
    # Define the files for the different variables
    msl_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_19961118${h}00.msl.nc"
    u10_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_19961118${h}00.10u.nc"
    v10_file="/badc/ecmwf-era5/data/oper/an_sfc/1996/11/18/ecmwf-era5_oper_an_sfc_19961118${h}00.10v.nc"
    # Concatenate them into flist.txt
    echo ${msl_file}\;${u10_file}\;${v10_file} > flist.txt
    
    # Run DetectNodes for this time step
    DetectNodes \
        --in_data_list flist.txt \
        --out nodes/${h}.txt \
        --searchbymin "msl" \
        --outputcmd "msl,min,0;_VECMAG(u10,v10),max,2" \
        --mergedist "6.0" \
        --latname "latitude" --lonname "longitude" > logs/${h}.txt
done

In [ ]:
ls nodes

In [ ]:
# Visualise file
head nodes/00.txt

In [ ]:
# Clean
rm -rf logs flist.txt
# Do no remove nodes : We will use them next

### Minimal `StitchNodes`

In [ ]:
# Create file with list of node files you want to use
ls nodes/*.txt > flist.txt

In [ ]:
# Call minimal StitchNodes
StitchNodes \
--in_list flist.txt \
--in_fmt "lon,lat,slp,wind10" \
--out "tracks.csv" \
--out_file_format "csv"

In [ ]:
# Visualize the tracks file
head tracks.csv
# For each track it identified, a track_id was defined
# the csv contains information about the position in space and time for each point.